In [1]:
import pandas as pd

In [1]:
# Only use 7 columns (don't use observation count)
# filter for us birds
# Rename columns
# Drop nulls
# Clean up badly named birds
# convert date column to datetime object
# extract month from date
# map month column to season
# combine county and state

## Clean up for regions
# rename columns
# Strip leading whitespace from State
# drop numbers from regionName
# drop state from countyname
# Map alaska county names
# Capitalize city in St. Louis and Richmond works
# drop parish and county from CountyName

# merge dfs
# create pivot table of rarities
# county-region mapping dict
# state-county mapping dict
# list of birds, seasons, and states


In [ ]:
def clean_chunks(df):
    
    # filter for us birds
    
    # rename columns
    
    # drop nulls
    
    # clean badly named birds
    
    # get season thru date
    
    # combine county + state